# Terea 1: Reconocedor basado en whisper

# Imports

In [1]:
import os
import torch
import torch, torchaudio, glob
import soundfile as sf
import scipy.signal
import numpy as np
import random
import csv

from datasets import load_dataset

/home/giuseppe/.pyenv/versions/hlt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Consts

In [2]:
SEQ_LEN = 12

def seed_everything(seed):      
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(42)

### Data Download

In [3]:
dataset = load_dataset("FluidInference/musan", split="train")

if not os.path.exists("musan_small"):
    os.makedirs("musan_small", exist_ok=True)
    for i, example in enumerate(dataset):
        audio = example["audio"]["array"]
        sr = example["audio"]["sampling_rate"]
        sf.write(f"musan_small/file_{i}.wav", audio, sr)
        
url = 'https://openslr.elda.org/resources/28/rirs_noises.zip'

if not os.path.exists('RIRS_NOISES'):
    if not os.path.exists('rirs_noises.zip'):
        os.system('wget ' + url)
    os.system('unzip -q rirs_noises.zip')
    os.system('rm rirs_noises.zip')

Repo card metadata block was not found. Setting CardData to empty.


## 1.1
Aprovechando lo visto en los cuadernos del taller se propone crear un tokenizador adaptado al vocabulario de la tarea. <br>
La idea es que sea orientado a palabra ya que hay pocas y simplifica el reconocimiento. <br> 
Se puede reaprovechar el tokenizador
DigitSumTokenizer que aparece en el cuaderno: parte2/run11. Implemente el
tokenizador para la tarea: <br>
- fechas2/fechas2_train.es.csv <br>
- fechas2/fechas2_test.es.csv <br>

In [4]:
class WordTokenizer:
    def __init__(self, csv_file):
        self.word2index = {
            '<pad>': 0,
            '<sos>': 1,
            '<eos>': 2
        }
        self.index2word = {
            0: '<pad>',
            1: '<sos>',
            2: '<eos>'
        }

        self.build_vocab(csv_file)

    def build_vocab(self, csv_file):
        with open(csv_file, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                text = row['txt'].lower()
                for word in text.split():
                    if word not in self.word2index:
                        idx = len(self.word2index)
                        self.word2index[word] = idx
                        self.index2word[idx] = word

    def encode(self, text, seq_len=-1):
        tokens = ['<sos>'] + text.lower().split() + ['<eos>']
        ids = [self.word2index[w] for w in tokens]

        if seq_len > len(ids):
            ids += [self.word2index['<pad>']] * (seq_len - len(ids))

        return torch.tensor(ids)

    def decode(self, ids):
        if isinstance(ids, torch.Tensor):
            ids = ids.tolist()

        words = [self.index2word[i] for i in ids]
        text = ' '.join(words)
        text = text.replace('<sos>', '').replace('<eos>', '').replace('<pad>', '')
        return text.strip()


## 1.2
Reutilice el contenido del cuaderno part1/run5 para implementar un Trainset y Testset adaptados a la tarea fechas2 que estamos implementando. <br> 
Compruebe de forma similar a como se hace en el cuaderno varios ficheros de train con aumento de ruido aditivo y reverberación con figuras de sus espectrogramas y la posibilidad de escucharlos.

#### Augmentation Functions

In [5]:
class NoiseAug(object):
    def __init__(self, noise_dir='musan_small/', prob=0.5):
        self.prob = prob
        self.noises = glob.glob(noise_dir + '*.wav')

        if len(self.noises) == 0:
            print("[WARN] No noise files found, NoiseAug disabled.")
            self.prob = 0.0

    def __call__(self, x):
        if len(self.noises) == 0:
            return x

        if np.random.uniform() < self.prob:
            n = torchaudio.load(np.random.choice(self.noises))[0][0]

            if len(n) < len(x):
                n = torch.nn.functional.pad(n, (0, len(x)-len(n)))
            elif len(n) > len(x):
                t0 = np.random.randint(0, len(n) - len(x))
                n = n[t0:t0+len(x)]

            n = n.numpy()
            snr = np.random.uniform(5, 15)
            n = n * np.sqrt(x.std()**2 / (n.std()**2 + 1e-8)) * 10**(-snr/20)
            x = x + n

        return x

    
class RIRAug(object):
    def __init__(self, rir_dir='RIRS_NOISES/simulated_rirs', prob=0.5):
        self.prob = prob
        self.rirs = glob.glob(rir_dir + '/*.wav')

        if len(self.rirs) == 0:
            print("[WARN] No RIR files found, RIRAug disabled.")
            self.prob = 0.0

    def __call__(self, x):
        if len(self.rirs) == 0:
            return x

        if np.random.uniform() < self.prob:
            n = len(x)
            rir = torchaudio.load(np.random.choice(self.rirs))[0][0]
            rir = rir.numpy()
            rir = rir / (np.max(np.abs(rir)) + 1e-8)
            x = scipy.signal.convolve(x, rir)
            t0 = np.argmax(np.abs(rir))
            x = x[t0:t0+n]

        return x

#### Dataset Definitions

In [6]:
def identity(x):
    return x

class TrainDataset(torch.utils.data.Dataset):
    def __init__(
        self,
        csv_file,
        tokenizer,
        audio_len=4*16000,
        seq_len=SEQ_LEN,
        transform=[identity]
    ):
        self.audio_len = audio_len
        self.seq_len = seq_len
        self.transform = transform
        self.tokenizer = tokenizer

        with open(csv_file, encoding='utf-8') as f:
            reader = csv.DictReader(f)
            self.data = [(row['wav'], row['txt']) for row in reader]

        print(len(self.data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wav_path, text = self.data[idx]

        x, fs = torchaudio.load(wav_path)

        if x.shape[1] < self.audio_len:
            x = torch.nn.functional.pad(
                x, (0, self.audio_len - x.shape[1]), value=0
            )
        else:
            x = x[:, :self.audio_len]

        x = x[0].numpy()
        for t in self.transform:
            x = t(x)

        y = self.tokenizer.encode(text, self.seq_len)
        return x, y
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, csv_file, tokenizer, audio_len=4*16000, seq_len=SEQ_LEN):
        self.audio_len = audio_len
        self.seq_len = seq_len
        self.tokenizer = tokenizer

        with open(csv_file, encoding='utf-8') as f:
            reader = csv.DictReader(f)
            self.data = [(row['wav'], row['txt']) for row in reader]

        print(len(self.data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wav_path, text = self.data[idx]

        x, fs = torchaudio.load(wav_path)
        if x.shape[1] < self.audio_len:
            x = torch.nn.functional.pad(
                x, (0, self.audio_len - x.shape[1]), value=0
            )
        else:
            x = x[:, :self.audio_len]

        x = x[0]
        y = self.tokenizer.encode(text, self.seq_len)
        return x, y

In [7]:
tokenizer = WordTokenizer('fechas2/fechas2_train.es.csv')

trainset = TrainDataset(
    'fechas2/fechas2_train.es.csv',
    tokenizer,
    transform=[
        NoiseAug(prob=0.5),
        RIRAug(prob=0.5)
    ]
)

testset = TestDataset(
    'fechas2/fechas2_test.es.csv',
    tokenizer
)


[WARN] No RIR files found, RIRAug disabled.
10000
1000


## 1.3
Reutilice el tokenizador de la tarea 1.1, el trainset/testset de la tarea 1.2 y el contenido del cuaderno part3/run21 para implementar el entrenamiento y test de la tarea de fechas con los ficheros: fechas2_train.es.csv y fechas2_test.es.csv. <br>
Compruebe como se hace en el cuaderno la salida generada para algunos ejemplos de test y la visualización del modelo de atención y mida la tasa de error total WER en test. <br>
Si el modelo se entrena muy lento por falta de GPU no hace falta que se hagan muchas iteraciones completas de los datos, lo que se valora es el código. <br>
Puede usar también el frontend preentrenado que se muestra en el cuaderno parte3/run12 para mejorar las prestaciones.

In [8]:
class FeedForward(torch.nn.Module):
    def __init__(self, d_model=512, d_ff=1024, dropout=0.1, **kwargs):
        super().__init__()
        self.ff = torch.nn.Sequential(
            torch.nn.LayerNorm(d_model),
            torch.nn.Linear(d_model, d_ff),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout),
            torch.nn.Linear(d_ff, d_model),
        )
        
    def forward(self, x):
        return self.ff(x)

class SelfAttention(torch.nn.Module):
    def __init__(self, d_model, n_heads=8, d_head=64, dropout=0.1, **kwargs):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_head
        self.scale = torch.sqrt(torch.tensor(d_head, dtype=torch.float32))
        self.norm = torch.nn.LayerNorm(d_model)
        self.q_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.v_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.k_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(d_head*n_heads, d_model)

    def forward(self, x):
        x = self.norm(x)
        b = x.shape[0]
        q = self.q_linear(x).view(b, -1, self.n_heads, self.d_head)
        k = self.k_linear(x).view(b, -1, self.n_heads, self.d_head)
        v = self.v_linear(x).view(b, -1, self.n_heads, self.d_head) 

        scores = torch.einsum('bihd,bjhd->bhij', q, k) / self.scale
     
        att = scores.softmax(dim=-1)
        att = self.dropout(att)

        out = torch.einsum('bhij,bjhd->bihd', att, v).reshape(b, -1, self.n_heads*self.d_head)
        out = self.dropout(out)
        out = self.out(out)
        return out

class Encoder(torch.nn.Module):
    def __init__(self, nb_layers=6, d_model=256, max_len=1000, **kwargs):
        super().__init__()
        self.pos = torch.nn.Parameter(torch.randn(1, max_len, d_model))
        self.att = torch.nn.ModuleList([SelfAttention(d_model=d_model, **kwargs) for _ in range(nb_layers)])
        self.ff = torch.nn.ModuleList([FeedForward(d_model=d_model, **kwargs) for _ in range(nb_layers)])
        
    def forward(self, x):
        b, t, d = x.shape
        x = x + self.pos[:, :t, :]
        for att, ff in zip(self.att, self.ff):
            x = x + att(x)
            x = x + ff(x)
        return x


class CausalSelfAttention(torch.nn.Module):
    def __init__(self, d_model, n_heads=8, d_head=64, dropout=0.1, **kwargs):
        super().__init__()
        self.seq_len = kwargs['seq_len']
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_head
        self.scale = torch.sqrt(torch.tensor(d_head, dtype=torch.float32))
        self.norm = torch.nn.LayerNorm(d_model)
        self.q_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.v_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.k_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(d_head*n_heads, d_model)
        
        self.register_buffer("mask", torch.tril(torch.ones(self.seq_len, self.seq_len))[None, None, ...] == 0)
            

    def forward(self, x):
        x = self.norm(x)
        b, n, d = x.shape
        q = self.q_linear(x).view(b, -1, self.n_heads, self.d_head)
        k = self.k_linear(x).view(b, -1, self.n_heads, self.d_head)
        v = self.v_linear(x).view(b, -1, self.n_heads, self.d_head) 

        scores = torch.einsum('bihd,bjhd->bhij', q, k) / self.scale
        
        scores = scores.masked_fill(self.mask[:,:,:n,:n], float('-inf'))
        att = scores.softmax(dim=-1)
        att = self.dropout(att)

        out = torch.einsum('bhij,bjhd->bihd', att, v).reshape(b, -1, self.n_heads*self.d_head)
        out = self.dropout(out)
        out = self.out(out)
        return out

class CrossAttention(torch.nn.Module):
    def __init__(self, d_model, n_heads=8, d_head=64, dropout=0.1, **kwargs):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.d_head = d_head
        self.scale = torch.sqrt(torch.tensor(d_head, dtype=torch.float32))
        self.norm1 = torch.nn.LayerNorm(d_model)
        self.norm2 = torch.nn.LayerNorm(d_model)
        self.q_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.v_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.k_linear = torch.nn.Linear(d_model, d_head*n_heads)
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(d_head*n_heads, d_model)
    
    def forward(self, x1, x2):
        x1 = self.norm1(x1)
        x2 = self.norm2(x2)  
        b = x1.shape[0]
        q = self.q_linear(x1).view(b, -1, self.n_heads, self.d_head)
        k = self.k_linear(x2).view(b, -1, self.n_heads, self.d_head)
        v = self.v_linear(x2).view(b, -1, self.n_heads, self.d_head) 

        scores = torch.einsum('bihd,bjhd->bhij', q, k) / self.scale
     
        att = scores.softmax(dim=-1)
        att = self.dropout(att)

        out = torch.einsum('bhij,bjhd->bihd', att, v).reshape(b, -1, self.n_heads*self.d_head)
        out = self.dropout(out)
        out = self.out(out)
        return out, att
    
class Decoder(torch.nn.Module):
    def __init__(self, nb_layers=6, **kwargs):
        super().__init__()        
        self.seq_len = kwargs['seq_len']
        self.pos = torch.nn.Parameter(torch.randn(1, self.seq_len, kwargs['d_model']))
        self.att = torch.nn.ModuleList([CausalSelfAttention(**kwargs) for _ in range(nb_layers)])
        self.cross_att = torch.nn.ModuleList([CrossAttention(**kwargs) for _ in range(nb_layers)])
        self.ff = torch.nn.ModuleList([FeedForward(**kwargs) for _ in range(nb_layers)])
        
    def forward(self, x, enc):
        b, t, d = x.shape
        x = x + self.pos[:, :t, :]
        for att, cross_att, ff in zip(self.att, self.cross_att, self.ff):
            x = x + att(x)
            x = x + cross_att(x, enc)[0]
            x = x + ff(x)            
        return x

class SpecAug(torch.nn.Module):
    def __init__(self, prob_t_warp=0.5,
                       t_factor=(0.9, 1.1), 
                       f_mask_width = (0, 8), 
                       t_mask_width = (0, 10),
                       nb_f_masks=[1,2], 
                       nb_t_masks=[1,2] ):
        super().__init__()
        self.t_factor = t_factor
        self.f_mask_width = f_mask_width
        self.t_mask_width = t_mask_width
        self.nb_f_masks = nb_f_masks
        self.nb_t_masks = nb_t_masks
        self.prob_t_warp = prob_t_warp

    def time_warp(self, x):
        x = torch.nn.functional.interpolate(x, size=(int(x.shape[2]*np.random.uniform(*self.t_factor)), ))
        # print('warp', x.shape[2])
        return x
    
    def freq_mask(self, x):
        for _ in range(np.random.randint(*self.nb_f_masks)):
            f = np.random.randint(*self.f_mask_width)
            f0 = np.random.randint(0, x.shape[1]-f)
            # print('f', f0, f0+f)
            x[:,f0:f0+f,:] = 0
        return x

    def time_mask(self, x):
        for _ in range(np.random.randint(*self.nb_t_masks)):
            t = np.random.randint(*self.t_mask_width)
            t0 = np.random.randint(0, x.shape[2]-t)
            # print('t', t0, t0+t)
            x[:,:,t0:t0+t] = 0
        return x

    def forward(self, x):
        if np.random.uniform() < self.prob_t_warp:
            x = self.time_warp(x)
        x = self.freq_mask(x)
        x = self.time_mask(x)
        return x

In [9]:
class AudioFeatures(torch.nn.Module):
    def __init__(self, feat_dim=80, d_model=512, **kwargs):
        super().__init__()
        self.fe = torchaudio.transforms.MelSpectrogram(
                        n_fft=512, 
                        win_length=25*16, 
                        hop_length=10*16, 
                        n_mels=feat_dim)                            # 25ms window, 10ms shift
        self.spec_aug = SpecAug()
        self.linear = torch.nn.Linear(feat_dim, d_model)

    def forward(self, x): 
        x = self.fe(x)        
        x = (x+1e-6).log()
        if self.training:
            x = self.spec_aug(x)
        x = x.transpose(1, 2)
        x = self.linear(x)
        return x

class AudioTransformer(torch.nn.Module):
    def __init__(self, vocab_size=24, **kwargs):
        super().__init__()
        self.vocab_size = vocab_size
        self.seq_len = kwargs['seq_len']

        self.fe = AudioFeatures(**kwargs)
        self.enc = Encoder(**kwargs)
        
        self.emb = torch.nn.Embedding(vocab_size, kwargs['d_model'])
        self.dec = Decoder(**kwargs)
        self.out = torch.nn.Linear(kwargs['d_model'], vocab_size)
        

    def encoder(self, x):
        x = self.fe(x)
        return self.enc(x)

    def decoder(self, y, enc):
        y = self.emb(y)
        dec = self.dec(y, enc)
        return self.out(dec)
    
    def forward(self, x, y):
        enc = self.encoder(x)
        return self.decoder(y, enc)
                
    def loss(self, x, y):        
        logits = self(x, y[:,:-1])
        target = y[:,1:]
        loss = torch.nn.functional.cross_entropy(logits.reshape(-1, self.vocab_size), 
                                                 target.reshape(-1))
        return loss
    
           
    def generate(self, x):
        device = next(self.parameters()).device
        self.eval()
        y = [20,]
        with torch.no_grad():            
            enc = self.encoder(x.to(device))   
            
            while y[-1] != 22 and len(y) < self.seq_len:
                logits = self.decoder(torch.tensor(y).unsqueeze(0).to(device), enc)
                y.append(logits.argmax(-1)[:,-1].item())
                            
        return y
    

In [ ]:
vocab_size = len(tokenizer.word2index)

model = AudioTransformer(
    vocab_size=vocab_size,
    d_model=256,
    nb_layers=8,
    d_ff=512,
    n_heads=8,
    d_head=32,
    dropout=0.1,
    seq_len=SEQ_LEN
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
opt = torch.optim.Adam(model.parameters(), lr=3e-4)


nb_epochs = 5
batch_size = 32
model.train()

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
for e in range(nb_epochs):
    avg_loss = 0
    for x, y in trainloader:
        x = x.to(device)
        y = y.to(device)
        opt.zero_grad()
        loss = model.loss(x, y)
        loss.backward()
        opt.step()
        avg_loss += loss.item()
    print('epoch %d/%d: avg_loss: %.2f' % (e,nb_epochs,avg_loss/len(trainloader)))
       
torch.save([model, opt], 'model.pt')